Source: https://github.com/robertmartin8/PyPortfolioOpt/blob/master/cookbook/4-Black-Litterman-Allocation.ipynb 

# Downloading data

In [ ]:
# see install.md to get this working
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

# PyPortfolioOpt
from pypfopt import risk_models, expected_returns
from pypfopt import black_litterman
from pypfopt import BlackLittermanModel, plotting
from pypfopt import EfficientFrontier, objective_functions

from xgb import Trainer

In [ ]:
ETFs = ["SPY", "BND", "GLD", "HODL.PA"]
ohlc = yf.download(ETFs, period="15y")  # Open, High, Low, Close
closing_prices = ohlc["Close"]
closing_prices.tail()

# Shows closing prices for the ETFs the last 5 days.


In [ ]:
# Get closing prices, and show 5 latest ones. 
stock_prices = yf.download("SPY", period="15y")["Close"].dropna()
stock_prices.head()

In [ ]:
gold_prices = yf.download("GLD", period="15y")["Close"].dropna()
gold_prices.head()

In [ ]:
bond_prices = yf.download("BND", period="15y")["Close"].dropna()
bond_prices.head()

In [ ]:
crypto_prices = yf.download("HODL.PA", period="15y")["Close"].dropna()
crypto_prices.head()

In [ ]:
# Only fetches market caps for SPY and GLD

# mcaps = {}
# for t in ETFs:
#     try:
#         mcaps[t] = yf.Ticker(t).info["marketCap"]
#     except KeyError:
#         print(f"Inget market cap för {t}")
# mcaps

# Using hardcoded values instead:
mcaps = {
    "SPY": 481_714_784_600,     # manuellt från https://www.nasdaq.com/market-activity/etf/spy
    "BND": 122_936_437_056,     # manuellt från https://www.nasdaq.com/market-activity/etf/bnd
    "GLD": 77_500_318_000,      # manuellt från https://www.nasdaq.com/market-activity/etf/gld
    "HODL.PA": 1_200_295_500   #change to .PA     # manuellt från https://www.nasdaq.com/market-activity/etf/hodl
}
mcaps

# Constructing the prior

In [ ]:
# Covariance matrix between all assets
cov_matrix = risk_models.CovarianceShrinkage(closing_prices).ledoit_wolf()
cov_matrix

In [ ]:
# Risk aversion
delta = black_litterman.market_implied_risk_aversion(stock_prices)
print(delta)

In [ ]:
plotting.plot_covariance(cov_matrix, plot_correlation=True)

In [ ]:
# Prior implied returns (Pi), N×1 column vector
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, cov_matrix)
market_prior

In [ ]:
market_prior.plot.barh(figsize=(10,5))

# Views
Q contains the magnitude of each view, while P maps the views to the assets they belong to.

In [ ]:
# # Absolute views, this should be coming from XGBoost, placeholder values atm
# viewdict = {
#     "SPY": 0.08,      # t.ex. +8% förväntad avkastning
#     "BND": 0.03,      
#     "GLD": 0.01,      
#     "HODL": 0.20,
# }

# View confidence
Using Idzorek's method
 - Maybe we can use some error term to evaluate the confidence levels?

In [ ]:
# # Placeholder values:
# confidences = [0.6, 0.5, 0.3, 0.7]

In [ ]:
# # Using idzorek method (specifying our own confidence levels)
# # You can also create omega yourself by creating a diagonal matrix based on the views variances
# bl_idzorek = BlackLittermanModel(cov_matrix, pi=market_prior, absolute_views=viewdict, omega="idzorek", view_confidences=confidences)

In [ ]:
# # Posterior estimate of returns
# ret_bl = bl_idzorek.bl_returns()
# ret_bl

In [ ]:
# # Visualization how this compares to the prior and our views:
# rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(viewdict)], 
#              index=["Prior", "Posterior", "Views"]).T
# rets_df

In [ ]:
# rets_df.plot.bar(figsize=(12,8))
# # "Notice that the posterior is often between the prior and the views. 
# # This supports the fact that the BL method is essentially a Bayesian weighted-average of the prior and views,
# # where the weight is determined by the confidence."

In [ ]:
# # Posterior covariance estimate:
# cov_bl = bl_idzorek.bl_cov()
# plotting.plot_covariance(cov_bl)

# Portfolio allocation

In [ ]:
# from pypfopt import EfficientFrontier, objective_functions

In [ ]:
# ef = EfficientFrontier(ret_bl, cov_bl)
# #ef.add_objective(objective_functions.L2_reg)
# ef.max_sharpe() # to maximize sharpe ratio, can be changed
# weights = ef.clean_weights()
# weights

In [ ]:
# pd.Series(weights).plot.pie(figsize=(10,10))

In [ ]:
# # Example of how much to spend in each asset based on capital
# from pypfopt import DiscreteAllocation

# da = DiscreteAllocation(weights, closing_prices.iloc[-1], total_portfolio_value=10000)
# # alloc, leftover = da.lp_portfolio() funkar ej
# alloc, leftover = da.greedy_portfolio()
# print(f"Leftover: ${leftover:.2f}")
# alloc #antal andelar
df_crypto = pd.read_csv("merged2.csv", parse_dates=["Date"])

In [ ]:
import pandas as pd
from xgb import Trainer

SCALE   = 1.0          # =1 om du vill använda size oförändrat
trainer = Trainer.load("models")     # joblib-filen du sparade

viewdict, confidences = {}, []

for ticker in ETFs:                         # ["SPY", "BND", "GLD", "HODL"]
    if ticker == "HODL.PA":
        # ── läs merged2.csv, sortera, ta sista 8 rader ──────────────
        df_crypto = (
            pd.read_csv("merged2.csv", parse_dates=["Date"])
              .sort_values("Date")
              .tail(8)
        )
        action, size, confidence = trainer.predict(df_crypto)
        viewdict[ticker]  = size * SCALE
        confidences.append(confidence)
    else:
        viewdict[ticker]  = 0.0        # neutral vy
        confidences.append(0.0)        # låg vikt

viewdict, confidences


In [ ]:

# Black‑Litterman with Idzorek confidences
bl_idzorek = BlackLittermanModel(
    cov_matrix,
    pi=market_prior,
    absolute_views=viewdict,
    omega="idzorek",
    view_confidences=confidences,
)

ret_bl = bl_idzorek.bl_returns()
cov_bl = bl_idzorek.bl_cov()

# Efficient frontier
ef = EfficientFrontier(ret_bl, cov_bl)
ef.max_sharpe()
weights = ef.clean_weights()
weights

#OrderedDict([('BND', 0.1263),
             #('GLD', 0.07962),
            # ('HODL', 0.29918),
            # ('SPY', 0.49489)])

In [ ]:

pd.Series(weights).plot.pie(figsize=(8,8), autopct='%1.1f%%')


In [ ]:
%run Financial_data.ipynb

In [ ]:
# ────────────────────────────────────────────────────────────────
#  BLACK-LITTERMAN: bygg views från färdiga matriser (weekly)
# ────────────────────────────────────────────────────────────────
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import yfinance as yf
from pypfopt import risk_models, black_litterman, BlackLittermanModel, EfficientFrontier


# ▸ 0. SÄKERHETSKOLL  --------------------------------------------------


missing = [name for name in needed if name not in globals()]
if missing:
    raise RuntimeError(
        f"Variabler saknas i minnet: {missing}. "
        "Kör först financial_data.ipynb så att de skapas."
    )

# ▸ 1. SKAPA closing_prices-matris från dina DataFrames  --------------
def to_series(df, name):
    s = df.set_index("Date")["Close"]
    s.index = pd.to_datetime(s.index)
    return s.rename(name)

closing_prices = pd.concat(
    [
        to_series(df_bnd,  "BND"),
        to_series(df_gld,  "GLD"),
        to_series(df_hodl, "HODL.PA"),
        to_series(df_spy,  "SPY"),
    ],
    axis=1,
).sort_index()
closing_prices


In [ ]:

# ▸ 2. WEEKLY views + confidences  ------------------------------------
def weekly_views(sent_df, price_df):
    sent_w = sent_df.assign(Date=pd.to_datetime(sent_df["Date"]))\
                    .set_index("Date")\
                    .resample("W-FRI").last().dropna()
    px_w   = price_df.resample("W-FRI").last()

    colmap = {"SPY_FGI":"SPY", "BND_FGI":"BND",
              "GLD_FGI":"GLD", "Crypto_FGI":"HODL.PA"}
    views, conf = {}, []
    for scol, tic in colmap.items():
        fgi  = sent_w[scol]
        prc  = px_w[tic].loc[fgi.index]
        fwd  = np.log(prc.shift(-1)/prc)          # 1-veckas log-retur
        tmp  = pd.concat({"cls":fgi,"fwd":fwd}, axis=1).dropna()
        cls  = int(fgi.iloc[-1])                  # senaste klass
        views[tic] = tmp.groupby("cls")["fwd"].mean().get(cls, 0.0)
        conf.append(1 / (tmp.query("cls==@cls")["fwd"].var() + 1e-8))
    conf = (np.array(conf)/np.max(conf)).tolist()
    return views, conf

viewdict, confidences = weekly_views(merged.copy(), closing_prices)
print("Views:", viewdict)
print("Conf :", confidences)

# ▸ 3. BLACK-LITTERMAN  ------------------------------------------------
mcaps = {
    "SPY": 481_714_784_600,
    "BND": 122_936_437_056,
    "GLD":  77_500_318_000,
    "HODL.PA": 1_200_295_500,
}
cov   = risk_models.CovarianceShrinkage(closing_prices).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(closing_prices["SPY"])
pi    = black_litterman.market_implied_prior_returns(mcaps, delta, cov)

bl = BlackLittermanModel(
    cov,
    pi=pi,
    absolute_views=viewdict,
    omega="idzorek",
    view_confidences=confidences,
    risk_aversion=delta,
)
ef = EfficientFrontier(bl.bl_returns(), bl.bl_cov())
ef.max_sharpe()
weights = ef.clean_weights()

print("\nOPTIMERAD PORTFÖLJ (weekly BL):")
for k, v in weights.items():
    print(f"  {k:7s}: {v*100:5.1f} %")

# (valfritt) visa fördelningen
pd.Series(weights).plot.pie(figsize=(6,6), autopct='%1.1f%%')
plt.ylabel("")
plt.title("Black-Litterman allocation (veckovis)")
plt.show()
